# Bottom Up Tree Search Synthesizer and SMT Based Verification Oracle

The Bottom Up Tree Search Synthesizer accepts a CFG written in z3 expressions and synthesizes z3 expressions, and the SMT Based Verification Oracle operates on z3 expressions.

In [1]:
from z3 import *

from iterate_z3_components_in_z3_expr_ref_in_reverse_polish_notation import iterate_z3_components_in_z3_expr_ref_in_reverse_polish_notation


z3_S = Int('S')
z3_B = Bool('B')
z3_x = Int('x')
z3_y = Int('y')

z3_non_terminals = { z3_S, z3_B }
z3_terminals = { z3_x, z3_y }
# non_terminals to a set of tuples containing the production_rule in reverse_polish_notation
z3_production_rules = {
  z3_S: {
    # x
    tuple(iterate_z3_components_in_z3_expr_ref_in_reverse_polish_notation(z3_x)),
    # y
    tuple(iterate_z3_components_in_z3_expr_ref_in_reverse_polish_notation(z3_y)),
    # 0
    tuple(iterate_z3_components_in_z3_expr_ref_in_reverse_polish_notation(IntVal(0))),
    # 1
    tuple(iterate_z3_components_in_z3_expr_ref_in_reverse_polish_notation(IntVal(1))),
    # + S S
    tuple(iterate_z3_components_in_z3_expr_ref_in_reverse_polish_notation(z3_S + z3_S)),
    # - S S
    tuple(iterate_z3_components_in_z3_expr_ref_in_reverse_polish_notation(z3_S - z3_S)),
    # ite B S S
    tuple(iterate_z3_components_in_z3_expr_ref_in_reverse_polish_notation(If(z3_B, z3_S, z3_S))),
  },
  z3_B: {
    # and B B
    tuple(iterate_z3_components_in_z3_expr_ref_in_reverse_polish_notation(And(z3_B, z3_B))),
    # or B B
    tuple(iterate_z3_components_in_z3_expr_ref_in_reverse_polish_notation(Or(z3_B, z3_B))),
    # not B
    tuple(iterate_z3_components_in_z3_expr_ref_in_reverse_polish_notation(Not(z3_B))),
    # <= S S
    tuple(iterate_z3_components_in_z3_expr_ref_in_reverse_polish_notation(z3_S <= z3_S)),
    # = S S
    tuple(iterate_z3_components_in_z3_expr_ref_in_reverse_polish_notation(z3_S == z3_S)),
    # >= S S
    tuple(iterate_z3_components_in_z3_expr_ref_in_reverse_polish_notation(z3_S >= z3_S)),
  }
}
z3_start_symbol = z3_S


z3_input_variable_list = [z3_x, z3_y]
z3_function_declaration = max2 = Function('max2', IntSort(), IntSort(), IntSort())
z3_constraint = And(max2(z3_x, z3_y) >= z3_x, max2(z3_x, z3_y) >= z3_y, Or(max2(z3_y, z3_x) == z3_x, max2(z3_y, z3_x) == z3_y))

## Initializing `z3_verification_oracle_instance`

In [2]:
from z3_verification_oracle import z3_verification_oracle


z3_verification_oracle_instance = z3_verification_oracle(z3_input_variable_list, z3_function_declaration, z3_constraint)
next(z3_verification_oracle_instance)

## Initializing `z3_bottom_up_tree_search_instance`

In [3]:
from z3_bottom_up_tree_search import z3_bottom_up_tree_search

z3_bottom_up_tree_search_instance = z3_bottom_up_tree_search(
    z3_non_terminals,
    z3_terminals,
    z3_production_rules,
    z3_start_symbol,
    z3_function_declaration,
    z3_constraint
)

## Program Synthesis and Verification

In [4]:
z3_expr_ref = next(z3_bottom_up_tree_search_instance)

In [5]:
print(f'z3_expr_ref {z3_expr_ref}')

z3_counterexample_input = z3_verification_oracle_instance.send(z3_expr_ref)

if z3_counterexample_input is not None:
    z3_expr_ref = z3_bottom_up_tree_search_instance.send(z3_counterexample_input)
    
print(f'z3_counterexample_input {z3_counterexample_input}')

z3_expr_ref x
z3_counterexample_input frozendict.frozendict({x: -1, y: 0})


In [6]:
print(f'z3_expr_ref {z3_expr_ref}')

z3_counterexample_input = z3_verification_oracle_instance.send(z3_expr_ref)

if z3_counterexample_input is not None:
    z3_expr_ref = z3_bottom_up_tree_search_instance.send(z3_counterexample_input)
    
print(f'z3_counterexample_input {z3_counterexample_input}')

z3_expr_ref 0
skipped 1
z3_counterexample_input frozendict.frozendict({x: -1, y: -1})


In [7]:
print(f'z3_expr_ref {z3_expr_ref}')

z3_counterexample_input = z3_verification_oracle_instance.send(z3_expr_ref)

if z3_counterexample_input is not None:
    z3_expr_ref = z3_bottom_up_tree_search_instance.send(z3_counterexample_input)
    
print(f'z3_counterexample_input {z3_counterexample_input}')

z3_expr_ref y
skipped 2*x
skipped 1 + x
skipped x + y
skipped 2
skipped 1 + y
skipped y + x
skipped 2*y
skipped -1 + x
skipped x + -1*y
skipped -1*x
skipped -1
skipped -1*y
skipped 1 + -1*x
skipped 1 + -1*y
skipped y + -1*x
skipped -1 + y
skipped 3*x
skipped 1 + 2*x
skipped 2*x + y
skipped 2 + x
z3_counterexample_input frozendict.frozendict({x: 0, y: -1})


In [8]:
print(f'z3_expr_ref {z3_expr_ref}')

z3_counterexample_input = z3_verification_oracle_instance.send(z3_expr_ref)

if z3_counterexample_input is not None:
    z3_expr_ref = z3_bottom_up_tree_search_instance.send(z3_counterexample_input)
    
print(f'z3_counterexample_input {z3_counterexample_input}')

z3_expr_ref 1 + x + y
skipped x + 2*y
skipped -1 + 2*x
skipped 2*x + -1*y
skipped 1 + x + -1*y
skipped -1 + x + y
skipped 3
skipped 2 + y
skipped 1 + y + x
skipped 1 + 2*y
skipped 2 + -1*x
skipped 2 + -1*y
skipped 1 + y + -1*x
skipped y + 2*x
skipped 2*y + x
skipped 3*y
skipped -1 + y + x
skipped 2*y + -1*x
skipped -1 + 2*y
skipped -1*x + y
skipped -1*y + x
skipped 1 + -1*x + y
skipped 1 + -1*y + x
skipped -2 + x
skipped -1 + x + -1*y
skipped x + -2*y
skipped -2*x
skipped -1 + -1*x
skipped -1*x + -1*y
skipped -2
skipped -1 + -1*y
skipped -1*y + -1*x
skipped -2*y
skipped 1 + -2*x
skipped 1 + -1*x + -1*y
skipped 1 + -1*y + -1*x
skipped 1 + -2*y
skipped y + -2*x
skipped -1 + y + -1*x
skipped -2 + y
skipped 4*x
skipped 1 + 3*x
skipped 3*x + y
skipped 2 + 2*x
skipped 1 + 2*x + y
skipped 2*x + 2*y
skipped -1 + 3*x
skipped 3*x + -1*y
skipped 1 + 2*x + -1*y
skipped -1 + 2*x + y
skipped 3 + x
skipped 2 + x + y
skipped 1 + x + 2*y
skipped 2 + x + -1*y
skipped x + 3*y
skipped -1 + x + 2*y
skipped

In [9]:
print(f'z3_expr_ref {z3_expr_ref}')

z3_counterexample_input = z3_verification_oracle_instance.send(z3_expr_ref)

if z3_counterexample_input is not None:
    z3_expr_ref = z3_bottom_up_tree_search_instance.send(z3_counterexample_input)
    
print(f'z3_counterexample_input {z3_counterexample_input}')

z3_expr_ref If(x >= 0, x, y)
skipped If(x >= 0, 0, x)
skipped If(x >= 0, 0, 1)
skipped If(x >= 0, 0, y)
skipped If(x >= 0, 1, x)
skipped If(x >= 0, 1, 0)
skipped If(x >= 0, 1, y)
skipped If(x >= 0, y, x)
skipped If(x >= 0, y, 0)
skipped If(x >= 0, y, 1)
skipped If(x >= 1, x, 0)
skipped If(x >= 1, x, 1)
skipped If(x >= 1, x, y)
skipped If(x >= 1, 0, x)
skipped If(x >= 1, 0, 1)
skipped If(x >= 1, 0, y)
skipped If(x >= 1, 1, x)
skipped If(x >= 1, 1, 0)
skipped If(x >= 1, 1, y)
skipped If(x >= 1, y, x)
skipped If(x >= 1, y, 0)
skipped If(x >= 1, y, 1)
skipped If(x >= y, x, 0)
skipped If(x >= y, x, 1)
z3_counterexample_input frozendict.frozendict({x: -1, y: -2})


In [10]:
print(f'z3_expr_ref {z3_expr_ref}')

z3_counterexample_input = z3_verification_oracle_instance.send(z3_expr_ref)

if z3_counterexample_input is not None:
    z3_expr_ref = z3_bottom_up_tree_search_instance.send(z3_counterexample_input)
    
print(f'z3_counterexample_input {z3_counterexample_input}')

z3_expr_ref If(x >= y, x, y)
z3_counterexample_input None
